# **Homework 2-1 Phoneme Classification**

## The DARPA TIMIT Acoustic-Phonetic Continuous Speech Corpus (TIMIT)
The TIMIT corpus of reading speech has been designed to provide speech data for the acquisition of acoustic-phonetic knowledge and for the development and evaluation of automatic speech recognition systems.

This homework is a multiclass classification task, 
we are going to train a deep neural network classifier to predict the phonemes for each frame from the speech corpus TIMIT.

link: https://academictorrents.com/details/34e2b78745138186976cbc27939b1b34d18bd5b3

## Download Data
Download data from google drive, then unzip it.

You should have `timit_11/train_11.npy`, `timit_11/train_label_11.npy`, and `timit_11/test_11.npy` after running this block.<br><br>
`timit_11/`
- `train_11.npy`: training data<br>
- `train_label_11.npy`: training label<br>
- `test_11.npy`:  testing data<br><br>

**notes: if the google drive link is dead, you can download the data directly from Kaggle and upload it to the workspace**




In [1]:
# !gdown --id '1HPkcmQmFGu-3OknddKIa5dNDsR05lIQR' --output data.zip
# !unzip data.zip
# !ls 

In [2]:
from tqdm import tqdm

In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

## Preparing Data
Load the training and testing data from the `.npy` file (NumPy array).

In [4]:
import numpy as np

print('Loading data ...')

data_root='./timit_11/'
train = np.load(data_root + 'train_11.npy')
train_label = np.load(data_root + 'train_label_11.npy')
test = np.load(data_root + 'test_11.npy')

print('Size of training data: {}'.format(train.shape))
print('Size of testing data: {}'.format(test.shape))

Loading data ...
Size of training data: (1229932, 429)
Size of testing data: (451552, 429)


## Create Dataset

In [5]:
import torch
from torch.utils.data import Dataset

class TIMITDataset(Dataset):
    def __init__(self, X, y=None, mean=None, std=None):
        self.data = torch.from_numpy(X).float()
        if y is not None:
            y = y.astype(np.int)
            self.label = torch.LongTensor(y)
        else:
            self.label = None
        
        if std is not None:
            self.mean = mean
            self.std = std
        else:
            self.mean = self.data.mean(dim=0, keepdim=True)
            self.std = self.data.std(dim=0, keepdim=True)
        
        self.data = (self.data-self.mean)/self.std

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)


Split the labeled data into a training set and a validation set, you can modify the variable `VAL_RATIO` to change the ratio of validation data.

In [6]:
VAL_RATIO = 0.2

percent = int(train.shape[0] * (1 - VAL_RATIO))
train_x, train_y, val_x, val_y = train[:percent], train_label[:percent], train[percent:], train_label[percent:]
print('Size of training set: {}'.format(train_x.shape))
print('Size of validation set: {}'.format(val_x.shape))

Size of training set: (983945, 429)
Size of validation set: (245987, 429)


Create a data loader from the dataset, feel free to tweak the variable `BATCH_SIZE` here.

In [7]:
BATCH_SIZE = 128

from torch.utils.data import DataLoader

train_set = TIMITDataset(train_x, train_y)
val_set = TIMITDataset(val_x, val_y, mean=train_set.mean, std=train_set.std)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True) #only shuffle the training data
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False)

Cleanup the unneeded variables to save memory.<br>

**notes: if you need to use these variables later, then you may remove this block or clean up unneeded variables later<br>the data size is quite huge, so be aware of memory usage in colab**

In [8]:
import gc

del train, train_label, train_x, train_y, val_x, val_y
gc.collect()

232

## Create Model

Define model architecture, you are encouraged to change and experiment with the model architecture.

In [17]:
import torch
import torch.nn as nn

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()

        n = [429, 1024]
        n += [512 for i in range(20)]
        n += [256, 128]
        self.n = n
        self.l_n = len(n)-1

        self.bn0 = nn.BatchNorm1d(n[0])
        self.bn = nn.ModuleList([nn.BatchNorm1d(i) for i in n[1:]])
        self.linear = nn.ModuleList([nn.Linear(n[i], n[i+1]) for i in range(len(n)-1)])
        self.out = nn.Linear(n[-1], 39) 

        self.act_fn = nn.SELU()
        self.relu = nn.ReLU()
        self.drop1 = nn.Dropout(0.1)

    def forward(self, x):

        # x = self.bn0(x)

        for i in range(self.l_n-1):
            x = self.linear[i](x)
            x = self.act_fn(x)
            x = self.bn[i](x)
            x = self.drop1(x)

        x = self.linear[-1](x)
        x = self.act_fn(x)
        x = self.out(x)
        
        return x

## Training

In [18]:
#check device
def get_device():
  return 'cuda' if torch.cuda.is_available() else 'cpu'

Fix random seeds for reproducibility.

In [19]:
# fix random seed
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

Feel free to change the training parameters here.

In [21]:
# fix random seed for reproducibility
same_seeds(52728)

# get device 
device = get_device()
print(f'DEVICE: {device}')

# training parameters
num_epoch = 100               # number of training epoch
learning_rate = 3e-5       # learning rate

# the path where checkpoint saved
model_path = './model.ckpt'
overfit_model_path = './overfit_model.ckpt'

# create model, define a loss function, and optimizer
model = Classifier().to(device)
criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)

DEVICE: cuda


In [22]:
gain = 3/4 # for selu
for n, p in model.named_parameters():
    print(n, p.shape)
    if p.dim() >= 2:
        print("normal init")
        nn.init.xavier_normal_(p, gain)

bn0.weight torch.Size([429])
bn0.bias torch.Size([429])
bn.0.weight torch.Size([1024])
bn.0.bias torch.Size([1024])
bn.1.weight torch.Size([512])
bn.1.bias torch.Size([512])
bn.2.weight torch.Size([512])
bn.2.bias torch.Size([512])
bn.3.weight torch.Size([512])
bn.3.bias torch.Size([512])
bn.4.weight torch.Size([512])
bn.4.bias torch.Size([512])
bn.5.weight torch.Size([512])
bn.5.bias torch.Size([512])
bn.6.weight torch.Size([512])
bn.6.bias torch.Size([512])
bn.7.weight torch.Size([512])
bn.7.bias torch.Size([512])
bn.8.weight torch.Size([512])
bn.8.bias torch.Size([512])
bn.9.weight torch.Size([512])
bn.9.bias torch.Size([512])
bn.10.weight torch.Size([512])
bn.10.bias torch.Size([512])
bn.11.weight torch.Size([512])
bn.11.bias torch.Size([512])
bn.12.weight torch.Size([512])
bn.12.bias torch.Size([512])
bn.13.weight torch.Size([512])
bn.13.bias torch.Size([512])
bn.14.weight torch.Size([512])
bn.14.bias torch.Size([512])
bn.15.weight torch.Size([512])
bn.15.bias torch.Size([512])
bn

In [16]:
# start training

best_acc = 0.0
for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0

    # training
    model.train() # set the model to training mode
    for i, data in enumerate(tqdm(train_loader)):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad() 
        outputs = model(inputs) 
        batch_loss = criterion(outputs, labels)
        _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        batch_loss.backward() 
        optimizer.step() 

        train_acc += (train_pred.cpu() == labels.cpu()).sum().item()
        train_loss += batch_loss.item()

    # validation
    if len(val_set) > 0:
        model.eval() # set the model to evaluation mode
        with torch.no_grad():
            for i, data in enumerate(val_loader):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                batch_loss = criterion(outputs, labels) 
                _, val_pred = torch.max(outputs, 1) 
            
                val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
                val_loss += batch_loss.item()

            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
                epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader), val_acc/len(val_set), val_loss/len(val_loader)
            ))

            # if the model improves, save a checkpoint at this epoch
            if val_acc > best_acc:
                best_acc = val_acc
                torch.save(model.state_dict(), model_path)
                print('saving model with acc {:.3f}'.format(best_acc/len(val_set)))
    else:
        print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader)
        ))

# if not validating, save the last epoch
if len(val_set) == 0:
    torch.save(model.state_dict(), model_path)
    print('saving model at last epoch')
else:
    torch.save(model.state_dict(), overfit_model_path)
    print('saving overfit model')



  4%|▍         | 302/7688 [00:07<02:52, 42.74it/s]


KeyboardInterrupt: 

## Testing

Create a testing dataset, and load model from the saved checkpoint.

In [37]:
# create testing dataset
test_set = TIMITDataset(test, None, mean=train_set.mean, std=train_set.std)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

# create model and load weights from checkpoint
model = Classifier().to(device)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

Make prediction.

In [38]:
predict = []
model.eval() # set the model to evaluation mode
with torch.no_grad():
    for i, data in enumerate(test_loader):
        inputs = data
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability

        for y in test_pred.cpu().numpy():
            predict.append(y)

Write prediction to a CSV file.

After finish running this block, download the file `prediction.csv` from the files section on the left-hand side and submit it to Kaggle.

In [39]:
with open('prediction.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(predict):
        f.write('{},{}\n'.format(i, y))